<a href="https://colab.research.google.com/github/ETorresSacha/Analisis-de-datos/blob/main/analisis_datos_reto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import io
import os
from google.colab import files
import seaborn as sns

In [28]:

uploaded = files.upload()

Saving TEMP2.xlsx to TEMP2.xlsx


In [29]:
df = pd.read_excel(io.BytesIO(uploaded['TEMP2.xlsx']))

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [42]:
df.sort_values(by=['equipment','location','start_detention'],inplace=True)

In [44]:
df.reset_index(drop=True, inplace=True)

In [45]:
df.head()

,reason,equipment,location,start_detention,end_detention,pay_number,id,comment,Minutos
0,MANTENCION NO PROGRAMADA,C301,8A,2021-10-04 21:53:48,2021-10-04 23:00:00,NaN,2674096,Alarma freno traba y servicio aplicado,67
1,MANTENCION NO PROGRAMADA,C301,8A,2021-10-04 23:00:00,2021-10-05 11:00:00,NaN,0,NaN,720
2,MANTENCION NO PROGRAMADA,C301,8A,2021-10-05 11:00:00,2021-10-05 23:00:00,NaN,0,NaN,720
3,MANTENCION NO PROGRAMADA,C301,8A,2021-10-05 23:00:00,2021-10-06 08:48:37,NaN,0,NaN,588
4,MANTENCION NO PROGRAMADA,C301,AUX001,2020-06-24 12:14:16,2020-06-24 13:22:06,NaN,2091922,Filtros congelados,68


In [46]:
# Exportamos a excel los datos de mnp_caex
df.to_excel('mnp_caex.xlsx', index=False)


In [47]:
# Descargar el archivo
files.download('mnp_caex.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>